In [1]:
# Встановлення необхідних бібліотек
!pip install -q nltk ipywidgets

# Імпорт бібліотек
import re
import random
from collections import Counter
import numpy as np
import nltk
import ipywidgets as widgets
from IPython.display import display, clear_output

# Завантаження ресурсів NLTK
nltk.download('punkt', quiet=True)

print("Середовище готове.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.6 MB/s eta 0:00:00
Середовище готове.


In [4]:
# --- Крок 1: Завантаження та розархівування (цей крок потрібно дочекатися) ---

# 1. Завантажуємо ПОВНИЙ файл корпусу новин. Це може зайняти час.
!wget https://lang.org.ua/static/downloads/corpora/news.tokenized.shuffled.txt.bz2

# 2. Розархівовуємо завантажений файл
!bzip2 -dk news.tokenized.shuffled.txt.bz2

print("\nКорпус новин завантажено та розархівовано. Починаємо часткову обробку...")


# --- Крок 2: Часткова обробка даних ---

# Імпортуємо необхідні бібліотеки
import re
import random
from collections import Counter

# Допоміжні функції (вони залишаються без змін)
def count_words(tokenized_sentences):
    word_counts = Counter()
    for sentence in tokenized_sentences:
        word_counts.update(sentence)
    return word_counts

def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    word_counts = count_words(tokenized_sentences)
    closed_vocab = [word for word, cnt in word_counts.items() if cnt >= count_threshold]
    return closed_vocab

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = [token if token in vocabulary else unknown_token for token in sentence]
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

def preprocess_data(train_data, test_data, count_threshold):
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

# Функція, яка читає лише частину файлу
def load_and_process_partially(filepath, num_lines_to_process):
    tokenized_sentences = []
    print(f"Починаємо читати та обробляти перші {num_lines_to_process} рядків з файлу...")
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= num_lines_to_process:
                    break
                sentence = line.lower().strip()
                tokens = sentence.split()
                if tokens:
                    tokenized_sentences.append(tokens)
                if (i + 1) % 100000 == 0:
                    print(f"  Оброблено {i + 1} рядків...")
    except FileNotFoundError:
        print(f"Помилка: файл '{filepath}' не знайдено.")
        return []
    print(f"Завершено. Оброблено {len(tokenized_sentences)} речень.")
    return tokenized_sentences

LINES_TO_PROCESS = 200000
corpus_path = 'news.tokenized.shuffled.txt'

tokenized_data = load_and_process_partially(corpus_path, LINES_TO_PROCESS)

if tokenized_data:
    random.seed(42)
    random.shuffle(tokenized_data)
    train_size = int(len(tokenized_data) * 0.8)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    print(f"\nРозмір тренувальної вибірки: {len(train_data)} речень")
    print(f"Розмір тестової вибірки: {len(test_data)} речень")

    print("\nПопередня обробка та створення словника...")
    count_threshold = 2
    train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, count_threshold)

    print(f"Розмір словника: {len(vocabulary)} слів")
    print("\nПідготовка даних успішно завершена! Можна переходити до наступних кроків.")
else:
    print("Не вдалося обробити дані.")

--2025-06-28 14:07:34--  https://lang.org.ua/static/downloads/corpora/news.tokenized.shuffled.txt.bz2
Resolving lang.org.ua (lang.org.ua)... 65.21.91.242
Connecting to lang.org.ua (lang.org.ua)|65.21.91.242|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1107980123 (1.0G) [application/octet-stream]
Saving to: ‘news.tokenized.shuffled.txt.bz2.1’

news.tokenized.shuf 100%[===================>]   1.03G  30.6MB/s    in 35s     

2025-06-28 14:08:09 (30.1 MB/s) - ‘news.tokenized.shuffled.txt.bz2.1’ saved [1107980123/1107980123]

bzip2: Output file news.tokenized.shuffled.txt already exists.

Корпус новин завантажено та розархівовано. Починаємо часткову обробку...
Починаємо читати та обробляти перші 200000 рядків з файлу...
  Оброблено 100000 рядків...
  Оброблено 200000 рядків...
Завершено. Оброблено 200000 речень.

Розмір тренувальної вибірки: 160000 речень
Розмір тестової вибірки: 40000 речень

Попередня обробка та створення словника...
Розмір словника: 87381 слі

In [5]:
# --- Допоміжні функції для обробки даних ---

def count_words(tokenized_sentences):
    """Підраховує частоту кожного слова в корпусі."""
    word_counts = Counter()
    for sentence in tokenized_sentences:
        word_counts.update(sentence)
    return word_counts

def get_words_with_nplus_frequency(tokenized_sentences, count_threshold):
    """Отримує слова, які з'являються не менше count_threshold разів."""
    word_counts = count_words(tokenized_sentences)
    closed_vocab = [word for word, cnt in word_counts.items() if cnt >= count_threshold]
    return closed_vocab

def replace_oov_words_by_unk(tokenized_sentences, vocabulary, unknown_token="<unk>"):
    """Замінює слова, що не входять до словника, на токен unknown_token."""
    vocabulary = set(vocabulary)
    replaced_tokenized_sentences = []
    for sentence in tokenized_sentences:
        replaced_sentence = [token if token in vocabulary else unknown_token for token in sentence]
        replaced_tokenized_sentences.append(replaced_sentence)
    return replaced_tokenized_sentences

def preprocess_data(train_data, test_data, count_threshold):
    """Виконує повну попередню обробку даних."""
    vocabulary = get_words_with_nplus_frequency(train_data, count_threshold)
    train_data_replaced = replace_oov_words_by_unk(train_data, vocabulary)
    test_data_replaced = replace_oov_words_by_unk(test_data, vocabulary)
    return train_data_replaced, test_data_replaced, vocabulary

# --- Оптимізована функція для читання великого файлу ---

def load_and_process_partially(filepath, num_lines_to_process):
    tokenized_sentences = []
    print(f"Починаємо читати та обробляти перші {num_lines_to_process} рядків з файлу...")
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            for i, line in enumerate(f):
                if i >= num_lines_to_process:
                    break
                sentence = line.lower().strip()
                tokens = sentence.split()
                if tokens:
                    tokenized_sentences.append(tokens)
                if (i + 1) % 200000 == 0:
                    print(f"  Оброблено {i + 1} рядків...")
    except FileNotFoundError:
        print(f"Помилка: файл '{filepath}' не знайдено.")
        return []
    print(f"Завершено. Оброблено {len(tokenized_sentences)} речень.")
    return tokenized_sentences

# --- Основна частина запуску обробки ---

LINES_TO_PROCESS = 500000  # Можна налаштувати кількість рядків для обробки
corpus_path = 'news.tokenized.shuffled.txt'

tokenized_data = load_and_process_partially(corpus_path, LINES_TO_PROCESS)

if tokenized_data:
    random.seed(42) # для відтворюваності
    random.shuffle(tokenized_data)
    train_size = int(len(tokenized_data) * 0.8)
    train_data = tokenized_data[:train_size]
    test_data = tokenized_data[train_size:]

    print(f"\nРозмір тренувальної вибірки: {len(train_data)} речень")
    print(f"Розмір тестової вибірки: {len(test_data)} речень")

    print("\nПопередня обробка та створення словника...")
    count_threshold = 2
    train_data_processed, test_data_processed, vocabulary = preprocess_data(train_data, test_data, count_threshold)

    print(f"Розмір словника: {len(vocabulary)} слів")
    print("\nКрок 2 успішно завершено!")
else:
    print("Не вдалося обробити дані.")

Починаємо читати та обробляти перші 500000 рядків з файлу...
  Оброблено 200000 рядків...
  Оброблено 400000 рядків...
Завершено. Оброблено 500000 речень.

Розмір тренувальної вибірки: 400000 речень
Розмір тестової вибірки: 100000 речень

Попередня обробка та створення словника...
Розмір словника: 151073 слів

Крок 2 успішно завершено!


In [7]:
def count_n_grams(data, n, start_token='<s>', end_token='</s>'):
    """Підраховує кількість n-грам у даних."""
    n_grams = Counter()
    for sentence in data:
        sentence = [start_token] * (n - 1) + sentence + [end_token]
        sentence = tuple(sentence)
        for i in range(len(sentence) - n + 1):
            n_gram = sentence[i:i+n]
            n_grams[n_gram] += 1
    return n_grams

# Обчислюємо N-грами різних розмірів
print("Обчислення N-грам...")
n_gram_counts_list = []
for n in range(1, 6): # Від уніграм (n=1) до 5-грам (n=5)
    print(f"Побудова {n}-грам...")
    n_grams = count_n_grams(train_data_processed, n)
    n_gram_counts_list.append(n_grams)

print("N-грамні моделі успішно побудовано.")

Обчислення N-грам...
Побудова 1-грам...
Побудова 2-грам...
Побудова 3-грам...
Побудова 4-грам...
Побудова 5-грам...
N-грамні моделі успішно побудовано.


In [8]:
def estimate_probability(word, previous_n_gram, n_gram_counts, n_plus1_gram_counts, vocabulary_size, k=1.0):
    """Оцінює ймовірність слова word, враховуючи попередню n-граму."""
    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts.get(previous_n_gram, 0)
    denominator = previous_n_gram_count + k * vocabulary_size
    n_plus1_gram = previous_n_gram + (word,)
    n_plus1_gram_count = n_plus1_gram_counts.get(n_plus1_gram, 0)
    numerator = n_plus1_gram_count + k
    probability = numerator / denominator
    return probability

def calculate_perplexity(sentence, n_gram_counts_list, vocabulary_size, k=1.0):
    """Обчислює перплексію для речення."""
    n = len(n_gram_counts_list)
    n_minus_1_gram_counts = n_gram_counts_list[n-2]
    n_gram_counts = n_gram_counts_list[n-1]

    sentence = ['<s>'] * (n - 1) + sentence + ['</s>']
    sentence = tuple(sentence)
    N = len(sentence)
    product_pi = 1.0

    for t in range(n - 1, N):
        previous_n_gram = sentence[t-(n-1):t]
        word = sentence[t]
        probability = estimate_probability(word, previous_n_gram, n_minus_1_gram_counts, n_gram_counts, vocabulary_size, k)

        if probability == 0:
            return float('inf')

        product_pi *= (1 / probability)

    perplexity = product_pi**(1/float(N))
    return perplexity

# Оцінимо перплексію нашої 5-грамної моделі на одному реченні
print("\nОцінка перплексії...")
if test_data_processed:
    example_sentence = test_data_processed[0]
    # Використовуємо 4-грами та 5-грами для розрахунку ймовірності 5-грамної моделі
    perplexity = calculate_perplexity(example_sentence, [n_gram_counts_list[3], n_gram_counts_list[4]], len(vocabulary))
    print(f"Речення: {' '.join(example_sentence)}")
    print(f"Перплексія для 5-грамної моделі: {perplexity:.4f}")
else:
    print("Тестові дані порожні, неможливо розрахувати перплексію.")


Оцінка перплексії...
Речення: всього до євро-2012 треба 14
Перплексія для 5-грамної моделі: 27498.1209


In [9]:
def get_suggestions(previous_tokens, n_gram_counts_list, vocabulary, k=1.0, start_with=None):
    """Отримує список найімовірніших наступних слів від різних N-грамних моделей."""
    model_suggestions = {}
    n_max = len(n_gram_counts_list)

    # Функція для оцінки ймовірностей для однієї N-грамної моделі
    def estimate_probabilities_for_model(prev_gram, n_minus_1_counts, n_counts, vocab, vocab_size, k_val):
        probabilities = {}
        for word in vocab:
            prob = estimate_probability(word, prev_gram, n_minus_1_counts, n_counts, vocab_size, k_val)
            probabilities[word] = prob
        return probabilities

    vocab_with_special_tokens = vocabulary + ["<unk>", "</s>"]
    vocabulary_size = len(vocab_with_special_tokens)

    # Ітеруємо від найдовших N-грам до найкоротших (backoff)
    for n in range(n_max, 1, -1):
        n_minus_1_counts = n_gram_counts_list[n-2]
        n_counts = n_gram_counts_list[n-1]

        # Визначаємо контекст
        context_len = n - 1
        previous_n_gram = previous_tokens[-context_len:] if len(previous_tokens) >= context_len else ["<s>"] * (context_len - len(previous_tokens)) + previous_tokens

        # Обчислюємо ймовірності
        probabilities = estimate_probabilities_for_model(tuple(previous_n_gram), n_minus_1_counts, n_counts, vocab_with_special_tokens, vocabulary_size, k)

        # Додаємо пропозиції, якщо вони ще не були додані від довшої моделі
        for word, prob in probabilities.items():
            if start_with and not word.startswith(start_with):
                continue
            if word not in model_suggestions:
                model_suggestions[word] = prob

    # Сортуємо слова за їхньою ймовірністю
    sorted_suggestions = sorted(model_suggestions.items(), key=lambda x: x[1], reverse=True)
    return sorted_suggestions

# Приклад використання
previous_text = "міністерство закордонних справ"
tokens = previous_text.split()
suggestions = get_suggestions(tokens, n_gram_counts_list, vocabulary, k=1.0)

print(f"\nПропозиції для тексту '{previous_text}':")
# Фільтруємо спец-токени перед виводом
filtered_suggestions = [s for s in suggestions if s[0] not in ['<unk>', '<s>', '</s>']]
for word, prob in filtered_suggestions[:5]:
    print(f"- {word} (ймовірність: {prob:.6f})")


Пропозиції для тексту 'міністерство закордонних справ':
- україни (ймовірність: 0.000172)
- росії (ймовірність: 0.000033)
- польщі (ймовірність: 0.000026)
- мзс (ймовірність: 0.000026)
- висловлює (ймовірність: 0.000013)


In [10]:
def run_interactive_console(n_gram_counts_list, vocabulary):
    """
    Запускає простий текстовий інтерфейс для тестування автозавершення.
    """
    print("\n--- Інтерактивний режим автозавершення ---")
    print("Введіть текст. Для виходу напишіть 'вихід' або 'exit'.")

    while True:
        try:
            text = input("\nВаш текст: ")
        except (KeyboardInterrupt, EOFError):
            print("\nЗавершення роботи.")
            break

        if text.lower().strip() in ['вихід', 'exit']:
            print("Завершення роботи.")
            break

        if not text:
            continue

        tokens = text.lower().strip().split()

        # Отримуємо пропозиції
        suggestions = get_suggestions(tokens, n_gram_counts_list, vocabulary, k=1.0)

        # Фільтруємо спец-токени ПЕРЕД виводом
        final_suggestions = [s for s in suggestions if s[0] not in ['<unk>', '<s>', '</s>']]

        if not final_suggestions:
            print("Немає пропозицій для даного контексту.")
        else:
            print("Найкращі пропозиції:")
            for word, prob in final_suggestions[:5]:
                print(f"- {word} (ймовірність: {prob:.6f})")

# Запуск інтерактивного інтерфейсу
# Переконуємось, що дані були успішно оброблені
if 'n_gram_counts_list' in locals() and 'vocabulary' in locals():
    run_interactive_console(n_gram_counts_list, vocabulary)
else:
    print("Помилка: моделі не були навчені. Перевірте виконання попередніх кроків.")


--- Інтерактивний режим автозавершення ---
Введіть текст. Для виходу напишіть 'вихід' або 'exit'.

Ваш текст: На даний момент
Найкращі пропозиції:
- у (ймовірність: 0.000060)
- в (ймовірність: 0.000046)
- на (ймовірність: 0.000026)
- проходить (ймовірність: 0.000026)
- відомо (ймовірність: 0.000026)

Ваш текст: В такому випадку рекомендовано
Найкращі пропозиції:
- проте (ймовірність: 0.000007)
- з (ймовірність: 0.000007)
- цими (ймовірність: 0.000007)
- явищами (ймовірність: 0.000007)
- за (ймовірність: 0.000007)

Ваш текст: вихід
Завершення роботи.
